In [1]:
cd ..

e:\GitHub\computervision


In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

from modules.models_util import EfficientNet
from modules.training_util import makeDataloader

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights_path = [
    'notebooks\efficientnet\models/2024-02-01 10-31-27/best.pth', # Fully trained 4k, 53.10%
    'notebooks\efficientnet\models/2024-02-01 09-55-37/best.pth', # Fully trained 2k, 52.74%
    'notebooks\efficientnet\models/2024-02-02 16-34-31/best.pth', # Fully trained even, 50.91%
    'notebooks\efficientnet\models/2024-31-01 18-53-33/best.pth', # 4k, 55.84%
    'notebooks\efficientnet\models/2024-31-01 19-19-52/best.pth' # 2k, 49.82%
]
model = EfficientNet.load(weights_path[4])
model.to(device)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [19]:
dataset_dir = 'E:\Datasets\cor-splits/temp-test'
dataloader = makeDataloader(dataset_dir, EfficientNet.data_transforms)

In [20]:
correct_predictions = 0
total_samples = 0

with torch.no_grad(): # no backtracking
    for inputs, labels in dataloader['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

accuracy = correct_predictions/total_samples
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 49.82%
